# Training, optimization and test of XGBoost for regression with different approaches

Content:

1. Importing and preparing data
2. Training a simple model without hyperparameter optimization and cross-validation
3. Basic hyperparameter optimization with cross-validation
4. Hyperparameter optimization with Optuna and cross-validation
5. Hyperparameter optimization with Optuna and cross-validation with stratification by bins
6. Nested cross-validation with Optuna for hyperparameter optimization

## Imports

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

import optuna

## Import and prepare data

In [ ]:
data_path = "path_to_your_pickled_dataframe"

features = ['Your feature column names go here']

target = "Your target column name goes here"

In [ ]:
df = pd.read_pickle(data_path)

X = df[features]
y = df[target].dropna()
X = X.loc[y.index]

In [ ]:
X.describe()

## Training a simple model without hyperparameter optimization and cross-validation

Workflow: Split data into train and test sets, scale data, train model, evaluate model.

In [ ]:
# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Fit only on training data
X_test = scaler.transform(X_test)

# Fit model
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

# Test model on test set
y_pred = xgb.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("RMSE: ", rmse)

## Basic hyperparameter optimization with cross-validation

With train_test_split and RandomizedSearchCV.  

Optuna is better/more state-of-the-art than RandomizedSearchCV.

In [ ]:
# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Fit only on training data
X_test = scaler.transform(X_test)

# The model to use
xgb = XGBRegressor()

# Generated by GitHub Copilot, maybe double-check ;)
params = {
    "n_estimators": [100, 500, 1000, 2000],
    "max_depth": [3, 5, 10, 20],
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "colsample_bytree": [0.3, 0.5, 0.7, 0.9, 1],
    "subsample": [0.3, 0.5, 0.7, 0.9, 1],
    "gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "min_child_weight": [0.5, 1, 3, 5, 7],
    "reg_alpha": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "reg_lambda": [0.1, 0.2, 0.3, 0.4, 0.5]
}

# Optimize XGBoost model using RandomizedSearchCV (using 100 iterations and 5-fold cross-validation)
xgb_cv = RandomizedSearchCV(xgb, params, n_iter=100, cv=5, scoring="neg_mean_squared_error", random_state=42, n_jobs=-1)

# Train model with best hyperparameters on all training data
xgb_cv.fit(X_train, y_train)

# Test model on test set
y_pred = xgb_cv.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"RMSE: {rmse}")

## Hyperparameter optimization with Optuna and cross-validation

With train_test_split, Optuna and cross_val_score.  

The is the "standard run" to use.

In [ ]:
# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Fit only on training data
X_test = scaler.transform(X_test)

In [ ]:
def objective(trial):

    # See https://github.com/optuna/optuna-examples/

    params = {
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster
        # "gblinear" also exists but doesn't work on the dataset
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]), 
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    if params["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)

        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)

        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # ---

    # Setup model with hyperparameters for this trial
    model = XGBRegressor(**params)

    # Cross-validation: Try different training/validation splits (5-fold CV)
    neg_rmses = cross_val_score(model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=5)
    neg_mean_rmse = neg_rmses.mean()

    return neg_mean_rmse

In [ ]:
# Hyperparameter tuning
study = optuna.create_study(direction="maximize") # Maximize negative RMSE = minimize RMSE
study.optimize(objective, n_trials=100, n_jobs=-1) # n_trials=100 -> 100 iterations, n_jobs=-1 -> use all CPU cores

# Get best hyperparameters
best_params = study.best_params

# Train model with best hyperparameters on all training data
xgb = XGBRegressor(**best_params)
xgb.fit(X_train, y_train)

# Test model on test set
y_pred = xgb.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
print(f"RMSE: {rmse}")

## Hyperparameter optimization with Optuna and cross-validation with stratification by bins

With train_test_split, Optuna and StratifiedKFold.  

Variation of the approach above. Only relevant for regression problems.

The y values are being split into bins (like in a histogram).  
Then for every split it is ensured that the distribution of the bins in the splits is as equal as possible/is preserved (stratification).  
This is important if the distribution of the values is not uniform.  
In the objective function you have to do the splits yourself with StratifiedKFold, because cross_val_score afaik does not support stratification.

In [ ]:
bins = pd.cut(y, bins=10, labels=np.arange(10))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=bins, random_state=42)

# Scale data
scaler = StandardScaler()
# Preserve structure of dataframes (needed later for stratified k-fold)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index) # Fit only on training data
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [ ]:
def objective(trial):

    # See https://github.com/optuna/optuna-examples/

    params = {
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster
        # "gblinear" also exists but doesn't work on the dataset
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]), 
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    if params["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)

        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)

        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # ---

    # Setup model with hyperparameters for this trial
    model = XGBRegressor(**params)

    # Cross-validation: Try different training/validation splits (5-fold CV)
    neg_rmses = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, valid_index in skf.split(X_train, bins.loc[X_train.index]):
        X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]

        model.fit(X_train_fold, y_train_fold)

        y_pred = model.predict(X_valid_fold)
        neg_rmses.append(-1 * mean_squared_error(y_valid_fold, y_pred, squared=False))

    neg_mean_rmse = np.mean(neg_rmses)

    return neg_mean_rmse

In [ ]:
# Hyperparameter tuning
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, n_jobs=-1)

# Get best hyperparameters
best_params = study.best_params

# Train model with best hyperparameters on all training data
xgb = XGBRegressor(**best_params)
xgb.fit(X_train, y_train)

# Test model on test set
y_pred = xgb.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
print(f"RMSE: {rmse}")

## Nested cross-validation with Optuna for hyperparameter optimization

With KFold, cross_val_score and Optuna.  

Nested cross-validation involves varying both the train-validation and train-test splits.  
This means there is an additional loop for different train-test splits.  
Nested cross-validation provides more robust metrics and allows us to assess the standard deviation of model performance.  
However, it also takes longer and is more computationally intensive.

In [ ]:
def objective(trial, X_train, y_train):

    # See https://github.com/optuna/optuna-examples/

    params = {
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster
        # "gblinear" also exists but doesn't work on the dataset
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]), 
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    if params["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)

        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)

        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # ---

    # Setup model with hyperparameters for this trial
    model = XGBRegressor(**params)

    # Inner cross-validation: Try different training/validation splits (5-fold CV)
    neg_rmses = cross_val_score(model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=5)
    neg_mean_rmse = neg_rmses.mean()

    return neg_mean_rmse

In [ ]:
outer_cv = KFold(5)

rmses = []

# Outer cross-validation loop: Try different train/test splits (5-fold CV)
for train_index, test_index in outer_cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Scale data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train) # Fit only on training data
    X_test = scaler.transform(X_test)

    # Optimize hyperparameters on the train set using Optuna
    study = optuna.create_study(direction="maximize")
    objective_fold = lambda trial: objective(trial, X_train, y_train) # Specialized objective function with X_train and y_train fixed
    study.optimize(objective_fold, n_trials=100, n_jobs=-1)

    # Train model on the full train set using the best hyperparameters found above
    best_params = study.best_params
    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train)

    # Evaluate model on the test set
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Record results for this fold/split
    rmses.append(rmse)

In [ ]:
print(f"RMSE: {np.mean(rmses)} +/- {np.std(rmses)}")